<a href="https://colab.research.google.com/github/JorgeAnsotegui/TFM/blob/main/Dataset_Tweaking/Renombrar_Yolo%26Coco_CorregirCategorias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
import os
import json
import re
import shutil
import yaml

In [1]:
# # Conectar Colab a Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
# Rutas de los archivos zip en Google Drive
ruta_zip_yolo = '/content/drive/MyDrive/TFM/Sin duplicados/YOLO/Polipos Final Images.v4-bboxdataaugmentation.yolov9.zip'
ruta_zip_coco = '/content/drive/MyDrive/TFM/Sin duplicados/COCO/Polipos Final Images.v4-bboxdataaugmentation.coco-segmentation.zip'

# Rutas de las carpetas destino
ruta_destino_yolo = '/content/dataset_yolo_4'
ruta_destino_coco = '/content/dataset_coco_4'

In [22]:
# Crear las carpetas destino si no existen
os.makedirs(ruta_destino_yolo, exist_ok=True)
os.makedirs(ruta_destino_coco, exist_ok=True)

# Descomprimir los archivos zip
with zipfile.ZipFile(ruta_zip_yolo, 'r') as zip_ref:
    zip_ref.extractall(ruta_destino_yolo)

with zipfile.ZipFile(ruta_zip_coco, 'r') as zip_ref:
    zip_ref.extractall(ruta_destino_coco)

print(f'Archivos descomprimidos en {ruta_destino_yolo} y {ruta_destino_coco}')

# Mover los archivos _annotations.coco.json a las carpetas correspondientes en YOLO
for split in ['train', 'test', 'valid']:
    coco_annotations_path = os.path.join(ruta_destino_coco, split, '_annotations.coco.json')
    yolo_annotations_path = os.path.join(ruta_destino_yolo, split, 'annotations.json')
    if os.path.exists(coco_annotations_path):
        shutil.move(coco_annotations_path, yolo_annotations_path)
        print(f'Movido {coco_annotations_path} a {yolo_annotations_path}')
    else:
        print(f'No se encontró {coco_annotations_path}')

# Función para renombrar archivos y actualizar anotaciones COCO
def renombrar_archivos_y_actualizar_anotaciones(carpeta_imagenes, carpeta_etiquetas, annotations_path, DataAugmentation):
    # Cargar anotaciones COCO
    with open(annotations_path, 'r') as f:
        coco_data = json.load(f)

    imagen_id_lookup = {image['file_name']: image['id'] for image in coco_data['images']}

    for nombre_archivo in os.listdir(carpeta_imagenes):
        if nombre_archivo.endswith('.jpg'):
            nuevo_nombre_base = re.sub(r'_jpg\.rf\.[a-f0-9]{32}', '', nombre_archivo)
            nuevo_nombre, ext = os.path.splitext(nuevo_nombre_base)

            ruta_actual_imagen = os.path.join(carpeta_imagenes, nombre_archivo)

            if DataAugmentation:
                contador = 1
                nuevo_nombre_con_numero = f"{nuevo_nombre}_{contador}{ext}"
                nueva_ruta_imagen = os.path.join(carpeta_imagenes, nuevo_nombre_con_numero)

                while os.path.exists(nueva_ruta_imagen):
                    contador += 1
                    nuevo_nombre_con_numero = f"{nuevo_nombre}_{contador}{ext}"
                    nueva_ruta_imagen = os.path.join(carpeta_imagenes, nuevo_nombre_con_numero)
            else:
                nueva_ruta_imagen = os.path.join(carpeta_imagenes, f"{nuevo_nombre}{ext}")

            # Renombrar la imagen
            os.rename(ruta_actual_imagen, nueva_ruta_imagen)

            # Renombrar el archivo de etiqueta correspondiente
            etiqueta_actual = os.path.join(carpeta_etiquetas, nombre_archivo.replace('.jpg', '.txt'))
            etiqueta_nueva = os.path.join(carpeta_etiquetas, nuevo_nombre_con_numero.replace('.jpg', '.txt') if DataAugmentation else f"{nuevo_nombre}{ext}".replace('.jpg', '.txt'))
            if os.path.exists(etiqueta_actual):
                os.rename(etiqueta_actual, etiqueta_nueva)

            # Actualizar el nombre de la imagen en las anotaciones COCO
            if nombre_archivo in imagen_id_lookup:
                image_id = imagen_id_lookup[nombre_archivo]
                for image in coco_data['images']:
                    if image['id'] == image_id:
                        image['file_name'] = nuevo_nombre_con_numero if DataAugmentation else f"{nuevo_nombre}{ext}"
                        break

    # Guardar las anotaciones COCO actualizadas
    with open(annotations_path, 'w') as f:
        json.dump(coco_data, f, indent=4)

    print(f'Archivos y anotaciones actualizadas en {carpeta_imagenes} y {carpeta_etiquetas}')

# Renombrar archivos y actualizar anotaciones COCO para cada split
for split in ['train', 'test', 'valid']:
    carpeta_imagenes = os.path.join(ruta_destino_yolo, split, 'images')
    carpeta_etiquetas = os.path.join(ruta_destino_yolo, split, 'labels')
    annotations_path = os.path.join(ruta_destino_yolo, split, 'annotations.json')
    if os.path.exists(carpeta_imagenes) and os.path.exists(carpeta_etiquetas) and os.path.exists(annotations_path):
        DataAugmentation = (split == 'train')  # Suponiendo que solo se usa DataAugmentation en el conjunto de entrenamiento
        renombrar_archivos_y_actualizar_anotaciones(carpeta_imagenes, carpeta_etiquetas, annotations_path, DataAugmentation)
    else:
        print(f'No se encontró la carpeta de imágenes, etiquetas o anotaciones para {split}')


Archivos descomprimidos en /content/dataset_yolo_4 y /content/dataset_coco_4
Movido /content/dataset_coco_4/train/_annotations.coco.json a /content/dataset_yolo_4/train/annotations.json
Movido /content/dataset_coco_4/test/_annotations.coco.json a /content/dataset_yolo_4/test/annotations.json
Movido /content/dataset_coco_4/valid/_annotations.coco.json a /content/dataset_yolo_4/valid/annotations.json
Archivos y anotaciones actualizadas en /content/dataset_yolo_4/train/images y /content/dataset_yolo_4/train/labels
Archivos y anotaciones actualizadas en /content/dataset_yolo_4/test/images y /content/dataset_yolo_4/test/labels
Archivos y anotaciones actualizadas en /content/dataset_yolo_4/valid/images y /content/dataset_yolo_4/valid/labels


# Arregla las categorias de Coco


In [23]:
# Función para cargar las categorías desde data.yaml de YOLO
def cargar_categorias_desde_yaml(ruta_yaml):
    with open(ruta_yaml, 'r') as f:
        data = yaml.safe_load(f)
        if 'names' in data:
            return data['names']
        else:
            return None

# Función para corregir las anotaciones COCO
def corregir_anotaciones_coco(annotations_path, nombres_categorias):
    with open(annotations_path, 'r') as f:
        coco_data = json.load(f)

    # Obtener las categorías actuales
    categorias = coco_data.get('categories', [])

    # Eliminar la categoría con id 0 si existe
    categorias = [cat for cat in categorias if cat['id'] != 0]

    # Verificar y corregir las supercategories
    for cat in categorias:
        if 'supercategory' not in cat or cat['supercategory'] != cat['name']:
            cat['supercategory'] = cat['name']

    # Verificar y corregir el orden y los nombres según nombres_categorias
    for i, nombre in enumerate(nombres_categorias, start=1):
        if i <= len(categorias):
            categorias[i-1]['name'] = nombre
        else:
            categorias.append({
                'id': i,
                'name': nombre,
                'supercategory': nombre
            })

    # Actualizar las categorías en coco_data
    coco_data['categories'] = categorias

    # Guardar las anotaciones COCO actualizadas
    with open(annotations_path, 'w') as f:
        json.dump(coco_data, f, indent=4)

    print(f'Anotaciones corregidas en {annotations_path}')

# Rutas de los archivos y carpetas relevantes
ruta_data_yaml = os.path.join(ruta_destino_yolo, 'data.yaml')

# Cargar nombres de categorías desde data.yaml de YOLO
nombres_categorias = cargar_categorias_desde_yaml(ruta_data_yaml)

if nombres_categorias:
    # Corregir anotaciones COCO para cada split
    for split in ['train', 'test', 'valid']:
        annotations_path = os.path.join(ruta_destino_yolo, split, 'annotations.json')
        if os.path.exists(annotations_path):
            corregir_anotaciones_coco(annotations_path, nombres_categorias)
        else:
            print(f'No se encontraron anotaciones en {annotations_path}')
else:
    print(f'No se pudieron cargar las categorías desde {ruta_data_yaml}')


Anotaciones corregidas en /content/dataset_yolo_4/train/annotations.json
Anotaciones corregidas en /content/dataset_yolo_4/test/annotations.json
Anotaciones corregidas en /content/dataset_yolo_4/valid/annotations.json


#Muevo los resultados a mi drive

In [24]:
import shutil
# Directorio de origen
src_dir = ruta_destino_yolo

# Directorio de destino
dest_dir = "/content/drive/MyDrive/TFM/Sin duplicados/Yolo&Coco/All_Data_Augmentation"

def copy_directory(src, dest):
    if not os.path.exists(dest):
        os.makedirs(dest)

    for root, dirs, files in os.walk(src):
        for name in dirs:
            src_dir = os.path.join(root, name)
            dest_dir = os.path.join(dest, os.path.relpath(src_dir, src))
            if not os.path.exists(dest_dir):
                os.makedirs(dest_dir)

        for name in files:
            src_file = os.path.join(root, name)
            dest_file = os.path.join(dest, os.path.relpath(src_file, src))
            if not os.path.exists(os.path.dirname(dest_file)):
                os.makedirs(os.path.dirname(dest_file))
            shutil.copy2(src_file, dest_file)

# Copiar el directorio de origen al directorio de destino
copy_directory(src_dir, dest_dir)

print(f"Directorio copiado de {src_dir} a {dest_dir}")

Directorio copiado de /content/dataset_yolo_4 a /content/drive/MyDrive/TFM/Sin duplicados/Yolo&Coco/All_Data_Augmentation
